<a href="https://colab.research.google.com/github/rafaeldjsm/Data_Science/blob/master/7DaysOfCode/7DaysOfCode_d1_CEAPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importado as Bibliotecas

In [1]:
!pip install wget &>/dev/null
import wget
import os
import pandas as pd
import numpy as np

# Baixando todo o dataset desde 2008 a 2022

In [2]:
# Url base
url0 = 'https://www.senado.leg.br/transparencia/LAI/verba/'


path = '/content/ceaps'

# Evitando criar um diretório já existente
if not os.path.isdir(path):
    !mkdir ceaps

ltceaps = os.listdir('/content/ceaps')

df = pd.DataFrame()

for k in range(2008,2023):
    elem = 'despesa_ceaps_'+ str(k) + '.csv'
    if elem not in ltceaps:
        # Baixando, caso não existam, os arquivos csv
        url = url0 + elem
        wget.download(url,out = path)
    dft = pd.read_csv('/content/ceaps/'+ elem, encoding="latin-1", sep = ';', header = 1, decimal=',')
    df = pd.concat([df,dft])


# Ajustando valores numéricos que vieram como strings e convertendo para float
df.VALOR_REEMBOLSADO = df.VALOR_REEMBOLSADO.apply(lambda x: str(x).replace(',','.').replace('\r\n',''))

df.VALOR_REEMBOLSADO = df.VALOR_REEMBOLSADO.astype('float')

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 310478 entries, 0 to 9296
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   ANO                310478 non-null  int64  
 1   MES                310478 non-null  int64  
 2   SENADOR            310478 non-null  object 
 3   TIPO_DESPESA       310478 non-null  object 
 4   CNPJ_CPF           307334 non-null  object 
 5   FORNECEDOR         307334 non-null  object 
 6   DOCUMENTO          286104 non-null  object 
 7   DATA               307334 non-null  object 
 8   DETALHAMENTO       176524 non-null  object 
 9   VALOR_REEMBOLSADO  310478 non-null  float64
 10  COD_DOCUMENTO      310476 non-null  float64
dtypes: float64(2), int64(2), object(7)
memory usage: 28.4+ MB


# Verificando campos com valores ausentes... interessante que os dados de fornecedor e CNPJ são os que têm mais dados faltantes... 🤔

In [10]:
df.isna().sum()

ANO                       0
MES                       0
SENADOR                   0
TIPO_DESPESA              0
CNPJ_CPF               3144
FORNECEDOR             3144
DOCUMENTO             24374
DATA                   3144
DETALHAMENTO         133954
VALOR_REEMBOLSADO         0
COD_DOCUMENTO             2
dtype: int64

# Qual senador teve maior número de despensas sem detalhar o fornecedor?

In [17]:
df_sen_falt = df[df.FORNECEDOR.isna()]

df_sen_falt.groupby('SENADOR').agg({'TIPO_DESPESA':'count','VALOR_REEMBOLSADO':'sum'}).sort_values(by='TIPO_DESPESA', ascending = False).rename(columns = {'TIPO_DESPESA':'Contagem'})

,Contagem,VALOR_REEMBOLSADO
SENADOR,,
CRISTOVAM BUARQUE,62,166231.24
MOZARILDO CAVALCANTI,59,207785.24
PATRÍCIA SABOYA GOMES,58,205838.38
IDELI SALVATTI,58,211363.29
OSMAR DIAS,57,196994.40
...,...,...
ADA MELLO,8,59962.46
MARIA DO CARMO ALVES,6,21472.23
ARTHUR VIRGÍLIO,6,43316.94


# Qual senador teve maior volume de despensas sem detalhar o fornecedor?

In [18]:
df_sen_falt.groupby('SENADOR').agg({'TIPO_DESPESA':'count','VALOR_REEMBOLSADO':'sum'}).sort_values(by='VALOR_REEMBOLSADO', ascending = False)

,TIPO_DESPESA,VALOR_REEMBOLSADO
SENADOR,,
JOSÉ AGRIPINO,37,214857.97
ROMEU TUMA,14,214621.82
PAULO PAIM,51,213600.62
ROSEANA SARNEY,44,213301.59
FRANCISCO DORNELLES,42,212230.09
...,...,...
MARCO ANTÔNIO COSTA,16,45354.64
ARTHUR VIRGÍLIO,6,43316.94
EDUARDO SUPLICY,36,39679.47


# Verificando por ano qual o senador teve maior gasto desde 2008?

In [3]:
df_gb_ano_sen = df.groupby(['SENADOR','ANO']).agg({'VALOR_REEMBOLSADO':'sum'}).sort_values(by='VALOR_REEMBOLSADO', ascending = False)
df_gb_ano_sen

,,VALOR_REEMBOLSADO
SENADOR,ANO,
EDUARDO AMORIM,2015,4389954.00
OMAR AZIZ,2019,531315.99
EDUARDO BRAGA,2018,529599.18
OMAR AZIZ,2021,527681.31
VANESSA GRAZZIOTIN,2013,521134.07
...,...,...
ORIOVISTO GUIMARÃES,2020,1548.02
OSVALDO SOBRINHO,2014,1281.28
JEFFERSON PRAIA,2011,1234.92


# Qual senador teve maior média anual de gastos?

In [6]:
df_gb_ano_sen.groupby('SENADOR').agg({'VALOR_REEMBOLSADO':'mean'}).sort_values(by='VALOR_REEMBOLSADO', ascending = False)

,VALOR_REEMBOLSADO
SENADOR,
EDUARDO AMORIM,692088.297778
GLADSON CAMELI,438852.100000
ROGÉRIO CARVALHO,433843.562500
VANESSA GRAZZIOTIN,430069.730000
PAULO ROCHA,424277.360000
...,...
IONE GUIMARÃES,11379.910000
WIRLANDE DA LUZ,11298.250000
SIQUEIRA CAMPOS,5926.990000


# Qual fornecedor teve maior volume de recurso recebido?

In [19]:
df.groupby('FORNECEDOR').agg({'VALOR_REEMBOLSADO':'sum'}).sort_values(by='VALOR_REEMBOLSADO', ascending = False)

,VALOR_REEMBOLSADO
FORNECEDOR,
LATAM,4793627.86
TAM,4482971.01
GOL,4461458.32
PROPAGTUR - Propag Turismo Ltda,4442760.27
ADRIA VIAGENS E TURISMO LTDA,2986537.37
...,...
Auto Posto Viracopos de Campinas Ltda,0.03
Posto 1400,0.03
GM PETROLEO,0.03


# Qual tipo de despesa teve maior volume de recurso demandado?

In [20]:
df.groupby('TIPO_DESPESA').agg({'VALOR_REEMBOLSADO':'sum'}).sort_values(by='VALOR_REEMBOLSADO', ascending = False)

,VALOR_REEMBOLSADO
TIPO_DESPESA,
"Locomoção, hospedagem, alimentação, combustíveis e lubrificantes",75159545.55
"Passagens aéreas, aquáticas e terrestres nacionais",72593209.14
"Aluguel de imóveis para escritório político, compreendendo despesas concernentes a eles.",58261630.81
"Contratação de consultorias, assessorias, pesquisas, trabalhos técnicos e outros serviços de apoio ao exercício do mandato parlamentar",52448082.87
Divulgação da atividade parlamentar,41060232.56
"Aquisição de material de consumo para uso no escritório político, inclusive aquisição ou locação de software, despesas postais, aquisição de publicações, locação de móveis e de equipamentos.",12320797.23
Serviços de Segurança Privada,3086757.10


# Qual ano teve maior gasto desde 2008?

In [21]:
df.groupby('ANO').agg({'VALOR_REEMBOLSADO':'sum'}).sort_values(by='VALOR_REEMBOLSADO', ascending = False)

,VALOR_REEMBOLSADO
ANO,
2015,28890550.61
2017,26672840.26
2018,25604701.33
2016,25301959.82
2019,25248242.40
2021,25020323.84
2013,24658952.86
2012,23354742.37
2014,22519394.12


# Qual mês teve maior gasto ?

In [22]:
df.groupby('MES').agg({'VALOR_REEMBOLSADO':'sum'}).sort_values(by='VALOR_REEMBOLSADO', ascending = False)

,VALOR_REEMBOLSADO
MES,
12,33459269.36
8,28787923.14
11,28417747.92
3,27718681.74
5,26693737.72
6,26403431.22
4,26233213.50
10,25550686.08
7,24453043.49


In [23]:
df.to_csv("ceaps_2008_2022")